In this notebook we read in the Master XML file made using NapaTrackMater and create N, Delta times Attribute dimensional vectors. N being the number of tracks present in the chosen region, R, Delta being the chosen time interval (t_minus + t_plus) {t - t_minus, t + t_plus} and Attributes being the morphological and the dynamic properties associated with cells in the tracks. We concatenate the Attribute componenets over the chosen time interval to create a Delta times Attribute dimensional vector and create a pandas dataframe with these vectors for further analysis.

In [ ]:
!pip install napatrackmater
!pip install git+https://github.com/kapoorlab/cellshape-helper
!pip install git+https://github.com/kapoorlab/cellshape-cloud
!pip install git+https://github.com/kapoorlab/cellshape-cluster    

In [1]:
%gui qt5
from napatrackmater.Trackvector import TrackVector
from pathlib import Path
from ipywidgets import interactive, widgets
from IPython.display import display
import napari 
from tifffile import imread
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
master_xml_path = Path('C:/Users/rando/Downloads/Mari_project/gt/master_gt_star_hyperstack.xml')
spot_csv_path = Path('C:/Users/rando/Downloads/Mari_project/gt/spots.csv')
track_csv_path = Path('C:/Users/rando/Downloads/Mari_project/gt/tracks.csv')
edges_csv_path = Path('C:/Users/rando/Downloads/Mari_project/gt/edges.csv')
image = imread('C:/Users/rando/Downloads/Mari_project/gt/rawk.tif')
show_tracks = False

shape_dynamic_dict = {'Time':0, 'Track ID':1,  'Dividing':2, 'Number_Dividing':3,  'Radius':4, 'Volume':5,'Eccentricity Comp First':6, 'Eccentricity Comp Second':7, 'Surface Area':8, 'Cluster Class':9, 'Speed':10, 'Directional_Change_Rate':11, 'Acceleration':12}

In [3]:
if show_tracks:
  viewer = napari.Viewer()
else:
    viewer = None
track_vectors = TrackVector(viewer,image,master_xml_path,spot_csv_path, track_csv_path, edges_csv_path, show_tracks = show_tracks)

Reading Master XML
Iterating over spots in frame
Iterating over tracks 547
getting attributes
obtianed spot attributes
obtained track attributes
obtained edge attributes


In [4]:
def track_setter(deltat, deltax, deltay):
    track_vectors.t_minus = deltat[0]
    track_vectors.t_plus = deltat[-1]
    
    track_vectors.x_start = deltax[0]
    track_vectors.x_end = deltax[-1]
    
    track_vectors.y_start = deltay[0]
    track_vectors.y_end = deltay[-1]
    

In [5]:
track_vector_widgets = interactive(track_setter, deltat = widgets.IntRangeSlider(
    value=[track_vectors.tstart, track_vectors.tend],
    min= track_vectors.tstart,
    max=track_vectors.tend,
    step=1,
    description='Delta Time',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
), 
    deltax = widgets.IntRangeSlider(
    value=[track_vectors.xmin, track_vectors.xmax],
    min= track_vectors.xmin,
    max=track_vectors.xmax,
    step=1,
    description='Delta X',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
), 
    deltay = widgets.IntRangeSlider(
    value=[track_vectors.ymin, track_vectors.ymax],
    min= track_vectors.ymin,
    max=track_vectors.ymax,
    step=1,
    description='Delta Y',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
),                               
                                  
                                  )

track_vector_widgets

interactive(children=(IntRangeSlider(value=(0, 265), continuous_update=False, description='Delta Time', max=26…

In [6]:
track_vectors._interactive_function()

returning shape and dynamic vectors as list 618


In [7]:
current_shape_dynamic_vectors = track_vectors.current_shape_dynamic_vectors

In [41]:
global_shape_dynamic_dataframe = []
for i in range(len(current_shape_dynamic_vectors)):
   
   vector_list = list(zip(current_shape_dynamic_vectors[i]))
   data_frame_list = np.transpose(np.asarray([vector_list[i] for i in range(len(vector_list))])[:,0,:]) 
   
   shape_dynamic_dataframe = pd.DataFrame(data_frame_list, columns =['Time', 'Track ID',  'Dividing', 'Number_Dividing',  'Radius', 'Volume', 'Eccentricity Comp First', 'Eccentricity Comp Second', 'Surface Area', 'Cluster Class', 'Speed', 'Directional_Change_Rate', 'Acceleration'])
   if len(global_shape_dynamic_dataframe) == 0:
        global_shape_dynamic_dataframe = shape_dynamic_dataframe
   else:
        global_shape_dynamic_dataframe = pd.concat([global_shape_dynamic_dataframe, shape_dynamic_dataframe],ignore_index=False)
   global_shape_dynamic_dataframe.set_index('Track ID')

In [42]:
global_shape_dynamic_dataframe.head()

,Time,Track ID,Dividing,Number_Dividing,Radius,Volume,Eccentricity Comp First,Eccentricity Comp Second,Surface Area,Cluster Class,Speed,Directional_Change_Rate,Acceleration
0,56.0,0.0,0.0,0.0,5.535109,746.0,2.161779,1.974284,1.138869,1.0,0.014182,32.186254,0.013397
1,3.0,0.0,0.0,0.0,5.912002,909.0,2.032241,1.730945,1.266064,1.0,0.020573,14.295329,0.005185
2,46.0,0.0,0.0,0.0,6.07443,986.0,2.527924,2.259719,1.38962,1.0,0.008633,66.5147,0.031786
3,33.0,0.0,0.0,0.0,6.648843,1293.0,2.192017,1.585482,1.596446,1.0,0.021719,129.498014,0.070329
4,20.0,0.0,0.0,0.0,8.011444,2262.0,1.92605,1.662763,2.353811,0.0,0.023955,65.897578,0.026411


In [43]:
global_shape_dynamic_dataframe.tail()

,Time,Track ID,Dividing,Number_Dividing,Radius,Volume,Eccentricity Comp First,Eccentricity Comp Second,Surface Area,Cluster Class,Speed,Directional_Change_Rate,Acceleration
4,263.0,547.0,0.0,0.0,6.330451,1116.0,3.256237,1.715262,1.499854,1.0,0.005027,40.214865,0.003399
5,264.0,547.0,0.0,0.0,6.055892,977.0,3.316175,1.530034,1.349856,1.0,0.00408,27.470563,0.00235
6,258.0,547.0,0.0,0.0,5.596262,771.0,2.9471,1.500575,1.151165,1.0,0.001954,0.0,0.0
7,261.0,547.0,0.0,0.0,6.42545,1167.0,3.294938,1.861812,1.557327,1.0,0.008141,8.981846,0.003519
8,257.0,547.0,0.0,0.0,5.629932,785.0,2.917116,1.452083,1.157277,1.0,0.0,0.0,0.0
